In [2]:
import pandas as pd
import os
from PIL import Image, ImageDraw
import shutil

### Code to create the masks for the data to be trained and tested

In [15]:
# set the project patch
my_path = "/Users/jiten/Masters/WorkPlace/"
folder_path = "/Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/"
csv_path_SAGIR = os.path.join(folder_path,'boundingInfo','SAGIR bounding boxes.csv')

image_dir = os.path.join(my_path, 'MRI_Image','06192023 SFI renamed')

source_folder_dict = {}
source_folder_ed_SAGIR = os.path.join(folder_path, 'SAGIR_edem_Images')
source_folder_frac_SAGIR = os.path.join(folder_path, 'SAGIR_frac_Images')
source_folder_dict['EDEMA'] = source_folder_ed_SAGIR
source_folder_dict['FRAC'] = source_folder_frac_SAGIR


train_dir_dict = {}
test_dir_dict = {}
val_dir_dict = {}

# #val_dir = os.path.join(folder_path, "validate")
# train_dir_SAGT1 = os.path.join(folder_path, "train_data_SAGT1")
# test_dir_SAGT1 = os.path.join(folder_path, "test_data_SAGT1")
# val_dir_SAGT1 = os.path.join(folder_path, "val_data_SAGT1")

train_dir_edema_SAGIR = os.path.join(folder_path, "train_data_edema_SAGIR")
test_dir_edema_SAGIR = os.path.join(folder_path, "test_data_edema_SAGIR")
val_dir_edema_SAGIR = os.path.join(folder_path, "val_data_edema_SAGIR")

train_dir_frac_SAGIR = os.path.join(folder_path, "train_data_frac_SAGIR")
test_dir_frac_SAGIR = os.path.join(folder_path, "test_data_frac_SAGIR")
val_dir_frac_SAGIR = os.path.join(folder_path, "val_data_frac_SAGIR")



# train_dir_dict['SAGT1'] = train_dir_SAGT1
train_dir_dict['EDEMA'] = train_dir_edema_SAGIR
train_dir_dict['FRAC'] = train_dir_frac_SAGIR

# test_dir_dict['SAGT1'] = test_dir_SAGT1
test_dir_dict['EDEMA'] = test_dir_edema_SAGIR
test_dir_dict['FRAC'] = test_dir_frac_SAGIR

# val_dir_dict['SAGT1'] = val_dir_SAGT1
val_dir_dict['EDEMA'] = val_dir_edema_SAGIR
val_dir_dict['FRAC'] = val_dir_frac_SAGIR

# get the dataframe for the bounding information
df_bounding = pd.read_csv(csv_path_SAGIR)

# create the data structure for mask directories
train_mask_dir_dict = {}
test_mask_dir_dict = {}
val_mask_dir_dict = {}

train_mask_edema_SAGIR = os.path.join(folder_path, "train_mask_edema_SAGIR")
test_mask_edema_SAGIR = os.path.join(folder_path, "test_mask_edema_SAGIR")
val_mask_edema_SAGIR = os.path.join(folder_path, "val_mask_edema_SAGIR")

train_mask_dir_dict['EDEMA'] = train_mask_edema_SAGIR
test_mask_dir_dict['EDEMA'] = test_mask_edema_SAGIR
val_mask_dir_dict['EDEMA'] = val_mask_edema_SAGIR

train_mask_frac_SAGIR = os.path.join(folder_path, "train_mask_frac_SAGIR")
test_mask_frac_SAGIR = os.path.join(folder_path, "test_mask_frac_SAGIR")
val_mask_frac_SAGIR = os.path.join(folder_path, "val_mask_frac_SAGIR")

train_mask_dir_dict['FRAC'] = train_mask_frac_SAGIR
test_mask_dir_dict['FRAC'] = test_mask_frac_SAGIR
val_mask_dir_dict['FRAC'] = val_mask_frac_SAGIR


In [4]:
def generateMask(img_width, img_height, x, y, w, h):
    """
    Function to generate the mask image 
    """

    # Create an empty image with a white background
    mask_image = Image.new("L", (img_width, img_height), 0)  # 'L' mode is for grayscale (0-255)

    # # Draw a rectangle on the mask image
    for i in range(len(x)):
        # Draw a rectangle on the mask image
        draw = ImageDraw.Draw(mask_image)
        draw.rectangle([x[i], y[i], x[i] + w[i], y[i] + h[i]], fill=255)  # 'fill' sets the color (255 for white)

    return mask_image

In [10]:
def createMask(df_bounding, source, dest, imgType):

    counter = 0 
    counter_self = 0

    if imgType == 'EDEMA':
        typeString = '[\'Edema\']'
    elif imgType == 'FRAC':
        typeString = '[\'Fracture\']'

    # filter the images based on the image type
    filter = (df_bounding['value_rectanglelabels'] == typeString)
    result_df = df_bounding[filter]

    # Get the list of all files in the folder
    all_files = os.listdir(source)

    #loop through the files and create the masks
    for file in all_files:

        #if file in result_df['image']:
        if (result_df['image'].eq(file)).any():

            condition = result_df['image'] == file
            #print(condition.sum())

            # initialize the array's to hold data
            img_width_arr = []
            img_height_arr = []
            x_arr = []
            y_arr = []
            w_arr = []
            h_arr = []

             # if there are multiple records then create multiple mask
            for i in range(condition.sum()):
                # get the image dimension
                img_width = result_df[condition].iloc[i]['original_width']
                img_height =  result_df[condition].iloc[i]['original_height']
                #print(len(img_width), file)
           
                img_width = int(img_width)  
                img_height = int(img_height)

                # get the coordinates for the bounding boxes
                x = result_df[condition].iloc[i]['value_x'] * img_width /100
                y = result_df[condition].iloc[i]['value_y'] * img_height /100
                w = result_df[condition].iloc[i]['value_width'] * img_width /100
                h = result_df[condition].iloc[i]['value_height'] * img_height /100
                #x,y,w,h = result_df.loc[file][['','value_y','','value_height']]

                img_width_arr.append(img_width)
                img_height_arr.append(img_height)
                x_arr.append(x)
                y_arr.append(y)
                w_arr.append(w)
                h_arr.append(h)

    
            # generate the mask
            mask_image = generateMask(img_width_arr[0], img_height_arr[0], x_arr, y_arr, w_arr, h_arr)

            # save the mask
            mask_image.save(f"{dest}/{file}")
            counter +=1
    
    print(f"Total Images:::{len(all_files)} ::::: of source :::: {source}")
    print(f"Total Mask created :::{counter} ::::: at destination :::: {dest}")


In [16]:
# create the masks for EDEMA images
imgType = 'EDEMA'

train_dir = train_dir_dict[imgType]
test_dir = test_dir_dict[imgType]
val_dir = val_dir_dict[imgType]

train_mask_dir = train_mask_dir_dict[imgType]
test_mask_dir = test_mask_dir_dict[imgType]
val_mask_dir = val_mask_dir_dict[imgType]

createMask(df_bounding,train_dir, train_mask_dir, imgType)

 # create the msk for testing data
createMask(df_bounding, test_dir, test_mask_dir , imgType)

# create the mask for validaiton data
createMask(df_bounding, val_dir, val_mask_dir, imgType)


Total Images:::559 ::::: of source :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/train_data_edema_SAGIR
Total Mask created :::559 ::::: at destination :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/train_mask_edema_SAGIR
Total Images:::185 ::::: of source :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/test_data_edema_SAGIR
Total Mask created :::185 ::::: at destination :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/test_mask_edema_SAGIR
Total Images:::116 ::::: of source :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/val_data_edema_SAGIR
Total Mask created :::116 ::::: at destination :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/val_mask_edema_SAGIR


In [17]:
# create the masks for FRACture images
imgType = 'FRAC'

train_dir = train_dir_dict[imgType]
test_dir = test_dir_dict[imgType]
val_dir = val_dir_dict[imgType]

train_mask_dir = train_mask_dir_dict[imgType]
test_mask_dir = test_mask_dir_dict[imgType]
val_mask_dir = val_mask_dir_dict[imgType]

createMask(df_bounding,train_dir, train_mask_dir, imgType)

 # create the msk for testing data
createMask(df_bounding, test_dir, test_mask_dir , imgType)

# create the mask for validaiton data
createMask(df_bounding, val_dir, val_mask_dir, imgType)

Total Images:::189 ::::: of source :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/train_data_frac_SAGIR
Total Mask created :::189 ::::: at destination :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/train_mask_frac_SAGIR
Total Images:::50 ::::: of source :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/test_data_frac_SAGIR
Total Mask created :::50 ::::: at destination :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/test_mask_frac_SAGIR
Total Images:::44 ::::: of source :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/val_data_frac_SAGIR
Total Mask created :::44 ::::: at destination :::: /Users/jiten/Masters/WorkPlace/MRI_MULTI_IMAGE_SEGMENTATION/val_mask_frac_SAGIR
